In [15]:
import pathlib
import pickle
import sys
from os import path
_up_one = path.abspath('../')
if _up_one not in sys.path:
    sys.path.insert(1, _up_one)
from lg_barycentric import get_matrix_vectors, LocalGroupBarycentric, mw_masses, m31_masses

from astropy.constants import G
from astropy.cosmology import Planck18_arXiv_v2 as cosmo
# from astropy.cosmology import WMAP9 as cosmo
import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.optimize import root

coord.galactocentric_frame_defaults.set('v4.0');

In [16]:
m31_cen = coord.SkyCoord(ra='00 42 44.330', dec='+41 16 07.50', distance=779*u.kpc,
                         unit=(u.hourangle, u.degree))

mw_cen = coord.Galactocentric(x=0*u.pc, y=0*u.pc, z=0*u.pc)

In [17]:
rho_c = 3 * cosmo.H(0.)**2 / (8*np.pi*G)
rho_c = rho_c.to(u.Msun/u.kpc**3)
rho_c

<Quantity 127.0528154 solMass / kpc3>

In [18]:
# mw_M = 1.3e12 * u.Msun
# m31_M = 2e12 * u.Msun

In [19]:
c = 10

In [33]:
models = dict()
for mw_M, m31_M in zip(mw_masses, m31_masses):
    mw_rvir = np.cbrt(mw_M / (200*rho_c) / (4/3*np.pi))
    m31_rvir = np.cbrt(m31_M / (200*rho_c) / (4/3*np.pi))
    
    mw_rs = mw_rvir / c
    m31_rs = m31_rvir / c
    
    print(mw_M / 1e12, m31_M / 1e12)
    print(f"{mw_rvir:.1f}, {m31_rvir:.1f}")
    print(f"{mw_rs:.2f}, {m31_rs:.2f}")
    print(m31_rs / mw_rs, mw_M / m31_M)
    
    lg_frame = LocalGroupBarycentric(mw_mass=mw_M, m31_mass=m31_M)
    print(m31_cen.transform_to(lg_frame).x.to(u.kpc),
          mw_cen.transform_to(lg_frame).x.to(u.kpc))
    
    models[f'{mw_M.value / 1e12:.1f}'] = {
        'mw_rvir': mw_rvir,
        'mw_rs': mw_rs,
        'mw_x': mw_cen.transform_to(lg_frame).x.to(u.kpc),
        'm31_rvir': m31_rvir,
        'm31_rs': m31_rs,
        'm31_x': m31_cen.transform_to(lg_frame).x.to(u.kpc),
        'lg_frame': lg_frame,
    }
    
    print('---\n')

0.9 solMass 2.3 solMass
203.7 kpc, 278.5 kpc
20.37 kpc, 27.85 kpc
1.3671886432341158 0.391304347826087
220.20247626324135 kpc -562.7396615539577 kpc
---

1.2 solMass 2.0 solMass
224.2 kpc, 265.9 kpc
22.42 kpc, 26.59 kpc
1.1856311014966876 0.6
293.6033016833228 kpc -489.3388361338763 kpc
---

1.5 solMass 1.7 solMass
241.5 kpc, 251.8 kpc
24.15 kpc, 25.18 kpc
1.0426036014588445 0.8823529411764706
367.00412710340424 kpc -415.9380107137949 kpc
---



In [35]:
cache_path = pathlib.Path('../cache')
cache_path.mkdir(exist_ok=True)

with open(cache_path / 'models.pkl', 'wb') as f:
    pickle.dump(models, f)

# MW:

In [38]:
def delta_c(c):
    fac = np.log(1+c) - c/(1+c)
    return 200/3 * c**3 / fac

def func(c, rho0):
    return (delta_c(c) * rho_c - rho0).decompose().value

From Wegg et al. 2019

In [45]:
rs = 27*u.kpc
rho_sun = 0.0092 * u.Msun / u.pc**3

In [46]:
R = 8.1*u.kpc
rho0 = rho_sun * (R/rs * (R/rs + 1)**2)
rho0

<Quantity 0.0046644 solMass / pc3>

In [47]:
res = root(func, 15., args=(rho0,))
mw_c = res.x[0]
mw_rvir = mw_c * rs
mw_c, mw_rvir

(9.217104283896367, <Quantity 248.86181567 kpc>)

In [48]:
mw_Mvir = 4*np.pi*rho0 * rs**3 * (np.log(1+mw_c) - mw_c/(1+mw_c))
mw_Mvir.to(1e12*u.Msun)

<Quantity 1.64050551 1e+12 solMass>